In [1]:
import csv # for writing dataframes to csv
import random # for making a random choice
import os # for scanning directories
import itertools
import string # for generating strings
from collections import Counter

import kintypes as kt # bringing large lists of kin types into the namespace
import math # for calculating logs
import pandas as pd
import numpy as np
import scipy.stats
import re
from tqdm import tqdm

testing = True # set to True to run code blocks with tests and examples
filtering = False # set to True to run the filtering process

# Internal co-selection

Internal co-selection refers to a process of kin term evolution whereby terminological changes in one part of the paradigm co-occur with changes in related parts of the paradigm, increasing the predictive structure of the paradigm.

In this notebook, we will build simulations to investigate the robustness of this tendency cross-linguistically, using data from Kinbank, a global database of kin terminology. 

We will measure internal co-selection in terms of the **mutual information** between Generation N and Generation N+1 in a particular kinship system. That tells us how much information can be gained from one generation by observing the other - how certain we can be about which children 'go with' which parents. This can be calculated as the **entropy** of one generation (how much unpredictable variation there is) minus the **conditional entropy** between the two generations (how much unpredictability remains in one generation after observing another).

## Calculating mutual information of a kinship system

To calculate the mutual information (MI) of a particular kinship system, we must perform the following steps:

1. Extract kin terminology data from Kinbank for this language.
2. Condense the full kinship system down to the terms we are interested in: Ego's generation and Ego's parents' generation.
3. Calculate the probabilities of each kin term within the generation in which it belongs; and the probabilities of each parent-child pair.
4. Calculate entropy, conditional entropy, subtract them from each other to get the mutual information of the system.

After we get that going, we can do these same calculations on simulated kinship systems.

### 1. Extract kin terminology from Kinbank

First, let's actually load our data in. The following function `get_kb_files()` pulls the full list of Kinbank filenames - one file per language. Later, we can iterate through these to generate MI values for every language in our dataset.

In [2]:
def get_kb_files(path) -> list:
    files = []
    directory = os.scandir(path)
    for file in directory:
        files.append(file.name)
    return files

all_kb_files = get_kb_files('../../kinbank')

In [3]:
all_kb_files

["'Are'are_area1240.csv",
 'Abui_abui1241.csv',
 'Achinese_achi1257.csv',
 'Achuar_achu1249.csv',
 'Aché_ache1246.csv',
 'Adang_adan1251.csv',
 'Adnyamathanha_adny1235.csv',
 'Aghul_aghu1253.csv',
 'Agob-Ende-Kawam_agob1244.csv',
 'Aguaruna_agua1253.csv',
 'Akhvakh_Northern_Akhvakh__akhv1239.csv',
 'Akkadian_akka1240.csv',
 'Aklanon_akla1241.csv',
 'Alamblak_alam1246.csv',
 'Alangan_alan1249.csv',
 'Albanian_alba1267.csv',
 'Albanian_Tosk__tosk1239.csv',
 'Alyawarr_alya1239.csv',
 'Amanab_aman1265.csv',
 'Amharic_amha1245.csv',
 'Amis_amis1246.csv',
 'Ampibabo_Lauje_ampi1237.csv',
 'Anambé_anam1249.csv',
 'Ancient_Greek_anci1242.csv',
 'Andi_andi1255.csv',
 'Anejom_anei1239.csv',
 'Angaite_anga1316.csv',
 'Aniwa_aniw1237.csv',
 'Anuta_anut1237.csv',
 'Arabana_arab1267.csv',
 'Arabana_arab1267a.csv',
 'Araki_arak1252.csv',
 'Araweté_araw1273.csv',
 'Archi_arch1244.csv',
 'Aria_(Mouk-Aria)_mouk1240a.csv',
 'Arizona_Yaqui_yaqu1251a.csv',
 'Armenian_nucl1235.csv',
 'Arosi_aros1241.csv',
 '

Using one of these filenames, we can extract the kin terminology from that file and populate a dictionary with it. We're only interested in two columns from the Kinbank data: `parameter`, which contains a short code indicating a **kin type**, and `word`, which contains the **kin term** associated with that kin type. An example of a row in the English data would be `mMeB, uncle`, where `mMeB` means 'male speaker's mother's older brother', and `uncle` is the term associated with that person.

In [4]:
def get_kin_terms(filepath: str) -> dict:
    ks = {}
    with open(filepath, encoding='utf8') as f:
        csv_reader = csv.DictReader(f)
        next(csv_reader) # to skip the header row
        for line in csv_reader:
            kin_type = line['parameter']
            kin_term = line['word']
            ks[kin_type] = kin_term
    return ks

### 2. Condense the system down

We're interested in the mutual information between the kin terms in Generation 0 (Ego's generaiton) and Generation +1 (Ego's parents' generation).

We want create a data structure that pairs up parent types with the corresponding child types. This is because we're interested in whether kinship systems maintain patterns of terminological distinctions and mergers across these two generations, so we will need to know which parent terms 'go with' which child terms.

In `kintypes`, you will find a list of pairs of kin types, where the first element in the pair is a parent type, and the second is their child; e.g. mMeB and mMeBD (mother's elder brother and mother's elder brother's daughter). 

`get_pairs()` takes a kinship system as input, and outputs a list of tuples. The first element in the tuple is the parent term, the second is the corresponding child term. 

In [5]:
def get_pairs(ks: dict) -> list:
    pairs_of_terms = []
    parent_types = []

    for pair in kt.ics_pairs:
        if pair[0] in ks and pair[1] in ks:
            pairs_of_terms.append((ks[pair[0]],ks[pair[1]]))
            parent_types.append(pair[0])
                
    return pairs_of_terms

But for our calculations, we'll still need to know which terms belong to which generation. Luckily, we know that the 0th element in each tuple is from Ego's parents' generation and the 1st element is from Ego's generation. So we can happily split these tuples down the middle and populate two lists with the terms. `split_pairs()` takes a list of pairs and sorts it into terms that belong to Ego's generation and terms that belong to Ego's parents' generation.

In [6]:
def split_pairs(pairs: list) -> list:
    gn = []
    gn1 = []
    for pair in pairs:
        gn.append(pair[1])
        gn1.append(pair[0])
    
    return gn,gn1

### 3. Calculate probabilities

To calculate entropy, we need a probability distribution over the terms in one single generation of a kinship system. So let's start with a function that can calculate the probability of a particular term.

Given a term and the full list of terms in the same generation, `probability()` counts how many times that term exists in `generation` and divides that by the total length of `generation`.

In [7]:
def probability(term: str, generation: list) -> float:
    return generation.count(term)/len(generation)

When calculating mutual information, we also need the **conditional entropy** between the two generations of our system. To calculate this, we will need not only the probabilities of terms in a generation, but also the **joint probabilities** of every pair of terms across those two generations. In other words, we need to calculate the probabilities of our `get_pairs` output.

Given two terms, `joint_probability()` counts how many pairs made of those two terms exist in `pairs`, then divides that by the total length of `pairs`.

In [8]:
def joint_probability(term1: str, term2: str, pairs: list) -> float:
    pair = (term1,term2)
    return pairs.count(pair)/len(pairs)

### 4. Calculating entropy and mutual information

Entropy (in bits) is defined as 

$$
H(X) = -\sum_{x \in X}p(x) log_2p(x)
$$

or, in English, it is the inverse sum over a distribution X of the probability of y * the log probability of y.

Entropy is a measure of the average level of uncertainty about the possible outcomes of a variable.

First, let's define a function `entropy()` that will iterate over a generation of the kinship system and output the entropy of that generation. 

Note: we only need one generation's entropy score to calculate mutual information - we make the arbitrary choice to calculate the entropy of Ego's parents' generation later in this notebook.

In [9]:
def entropy(generation: list) -> list:
    entropy = 0
    for term in set(generation): # using a set as we want to count each unique term only once
        p = probability(term,generation)
        #print('entropy of',term,p*math.log(p))
        entropy += p*math.log2(p)
    return round(-entropy,5)

Conditional entropy of Y given X is defined as

$$
H(Y|X) = -\sum_{x \in X,y \in Y}p(x,y) log_2 {p(x,y) \over p(x)}
$$

or in English, the inverse sum over two distributions Y and X of the probability of each y * the log probability of each y given x.

Conditional entropy is the amount of information needed to describe the outcome of a random variable Y given that we already know the value of another random variable X.

To calculate it, we need the joint probability of each pair (given by `joint_probability()`) and the probability of one member of that pair (given by `probability()`). We can then calculate the conditional probability of parent term given child term as the joint probability of those terms over the probability of the parent term.

`conditional_entropy()` iterates over all pairs to output the conditional entropy of Ego's generation given Ego's parents' generation.

In [10]:
def conditional_entropy(gn: list, pairs:list) -> float:
    entropy = 0
    for x,y in set(pairs): # x = parent, y = child
        p_xy = joint_probability(x,y,pairs)
        p_y = probability(y,gn)
        if p_xy > 0 and p_y > 0:
            #print('p(', x, '|', y,') = ', p_xy/p_y, 'p(y) = ', p_y)
            entropy += p_xy * math.log2(p_xy/p_y)
    return round(-entropy,5)

Finally, mutual information is defined as

$$
I(X;Y) \equiv H(X) - H(X|Y)
$$

or in English, entropy of X minus the conditional entropy of X given Y.

In this study, it is equal to the entropy of Ego's parents' generation minus the conditional entropy of Ego's parents' generation given Ego's generation. It tells us how much mutual dependence there is between these two generations; i.e. how much we can predict about one by observing the other.

In [11]:
def mutual_information(pairs: list):
    gn,gn1 = split_pairs(pairs)
    e = entropy(gn1)
    ce = conditional_entropy(gn,pairs)
    mi = e - ce
    return round(mi,5)

### 5. Calculate MI for each language and save data

With all of the above infrastructure, we can calculate the mutual information between G0 and G+1 for all the languages in Kinbank and save this data to a separate .csv file.

First, a function that calculates the relevant data and stores it.

In [12]:
def write_data(pairs,results):
    gn,gn1 = split_pairs(pairs)
    egn = entropy(gn)
    egn1 = entropy(gn1)
    ce = conditional_entropy(gn,pairs)
    mi = mutual_information(pairs)
    
    results['mutual_information'] = mi
    results['entropy_gn'] = egn
    results['entropy_gn1'] = egn1
    results['conditional_entropy'] = ce
    results['variation_gn'] = len(set(gn))
    results['variation_gn1'] = len(set(gn1))
    results['number_of_pairs'] = len(set(pairs))
    
    return results

In [13]:
families = ['Afro-Asiatic','Algic','Arawakan','Atlantic-Congo','Austroasiatic','Austronesian','Cariban','Dravidian','Indo-European',
            'Nakh-Daghestanian','Nuclear Trans New Guinea', 'Other','Pama-Nyungan','Pano-Tacanan','Salishan','Sino-Tibetan',
            'Tai-Kadai','Tupian','Turkic','Uralic','Uto-Aztecan']



In [14]:
def calculate_MI(families: list,filename: str):
    df = []
    codes = []
    filepath = '../../kinbank-family/'
    
    for family in families:
        all_files = get_kb_files(filepath + family)
        
        for file in all_files:
            print(file)
            match = re.search("[a-z]{4}[0-9]{4}[a-z]?", file)
            code = match.group()
            language = file.split('_' + code)[0]

            if code not in codes:
                codes.append(code)

                ks = get_kin_terms(filepath + family + '/' + file)

                pairs = get_pairs(ks)

                if pairs: # if pairs is not empty

                    results = {}
                    results['language'] = language
                    results['language_family'] = family
                    results['code'] = code
                    results['simulation_code'] = code + '_REAL'
                    results['simulation'] = 'N'
                    write_data(pairs,results)

                    df.append(results)
        
    pd.DataFrame(df).to_csv('../data/raw/' + filename + '.csv',index=False)
    
    return pd.DataFrame(df)

In [15]:
# calculate_MI(['Indo-European'],'test')

In [16]:
# calculate_MI(families,'kinbank_mi_UPDATED_NEW_KINTYPES')

Akkadian_akka1240.csv
Amharic_amha1245.csv
Hausa_haus1257.csv
Lebanese_Arabic_stan1323.csv
Old_Aramaic_olda1245.csv
Polci_polc1243.csv
Morgan1871_An-ah'-ne-lin_(Gros_Ventres_of_Prairie)_gros1243.csv
Morgan1871_Blood_Siksika_(Blood_Blackfoot)_bloo1239.csv
Morgan1871_Cree_of_the_Lowlands_cree1272.csv
Morgan1871_Delaware_comm1246.csv
Morgan1871_Etchemiu,_or_Malisete_male1293.csv
Morgan1871_Kaskaskia_algo1255.csv
Morgan1871_Kickapoo_(Kikapoo)_kick1244.csv
Morgan1871_Menomine_meno1252.csv
Morgan1871_Miami_miam1252.csv
Morgan1871_Miemac_mikm1235.csv
Morgan1871_Mohegan_mohe1244.csv
Morgan1871_Munsee_muns1251.csv
Morgan1871_Ojibwa_Kansas_ojib1241.csv
Morgan1871_Ojibwa_Lake_Huron_ojib1241.csv
Morgan1871_Ojibwa_Lake_Michigan_uppe1274.csv
Morgan1871_Ojibwa_Lake_Superior_ojib1241.csv
Morgan1871_Ottawa_(Otǎwa)_otta1242.csv
Morgan1871_Peoria_peor1238.csv
Morgan1871_Pianeshaw_miam1252.csv
Morgan1871_Piegan_Siksika_(Piegan_Blackfoot)_pieg1239.csv
Morgan1871_Plains_Cree_(Cree_of_the_Prairie)_plai1258.c

Tausug_(Sulu_Moro)_taus1251.csv
Tausug_taus1251a.csv
Tetum_(Tetun)_tetu1245.csv
Tetun_Dili_(Tetum)_tetu1246.csv
Thao_thao1240.csv
Tiale_tial1239.csv
Tialo_(Tomini)_tomi1243.csv
Tigak_tiga1245.csv
Tikopia_tiko1237.csv
Tikopia_tiko1237a.csv
Tirax_(Mae)_maee1241.csv
Tiri-Mea_(Grand_Couli)_tiri1258.csv
Tokelau_toke1240.csv
Tokelau_toke1240a.csv
Tokotu'a_(Kabaena)_kaba1285.csv
Tolaki_tola1247.csv
Tomadino_toma1248.csv
Tombulu_tomb1243.csv
Tongan_tong1325.csv
Tongarevans_penr1237.csv
Tonga_(Tonga_Islands)_tong1325.csv
Tonsea_tons1240.csv
Toqabaqita_toab1237.csv
Torete_tore1237.csv
Totoli_toto1304.csv
Tubuai_tubu1240.csv
Tukang_Besi_South_tuka1249.csv
Tulambatu_tula1253.csv
Tungag_tung1290.csv
Tuvalu_(Nanumea)_tuva1244.csv
Ulawa_ulaw1237.csv
Uma_Juman_(Kayan)_umaj1240.csv
Umiray_Dumaget_Agta_umir1236.csv
Unua_unua1237.csv
Urak_Lawoi_urak1238.csv
Ura_urav1235.csv
Uripiv-Wala-Rano-Atchin_urip1239.csv
Vaekau-Taumako_pile1238.csv
Vao_vaoo1237.csv
Varisi_vari1239.csv
Vera'a_vera1241.csv
Vures_vure

Pilagá_pila1245.csv
Puinave_puin1248.csv
Pumé_pume1238.csv
Páez_paez1247.csv
Qawasquar_qawa1238.csv
Quechan_quec1382.csv
Riantana_rian1263.csv
Rotokas_roto1249.csv
Sanapana_sana1298.csv
Savosavo_savo1255.csv
Sawi_sawi1257.csv
Selk_nam_onaa1245.csv
Seri_seri1257.csv
Setaman_Baktamin_seta1246.csv
Shasta_shas1239.csv
Sion_sion1247.csv
Siwai_Motuna_siwa1245.csv
Sougb_mani1235.csv
Southern_Coastal_Tsimshian_nucl1649.csv
Suki_suki1245.csv
Tabla_tabl1243.csv
Taiap_taia1239.csv
Tehuelche_tehu1242.csv
Teiwa_teiw1235.csv
Telefol_tele1256.csv
Tidore_(version_1)_tido1248.csv
Tidore_(version_2)_tido1248a.csv
Tidore_tido1249.csv
Tlingit_tlin1245.csv
Toba_toba1269.csv
Tolowa_tolo1259.csv
Touo_touo1238.csv
Trumai_trum1247.csv
Tsafiki_colo1249.csv
Tucanoan_Bará_waim1255.csv
Tucano_tuca1252.csv
Tuyuca_tuyu1244.csv
Tzeltal_tzel1254.csv
Umbu-Ungu_umbu1258.csv
Usurufa_usar1243.csv
Waffa_waff1241.csv
Wailaki_wail1244.csv
Wambaya_wamb1258.csv
Waorani_waor1240.csv
Wappo_wapp1239.csv
Wari__wari1268.csv
Warta_T

,language,language_family,code,simulation_code,simulation,mutual_information,entropy_gn,entropy_gn1,conditional_entropy,variation_gn,variation_gn1,number_of_pairs
0,Akkadian,Afro-Asiatic,akka1240,akka1240_REAL,N,0.00000,2.25000,1.00000,1.00000,5,2,10
1,Amharic,Afro-Asiatic,amha1245,amha1245_REAL,N,1.58497,2.25163,1.91830,0.33333,6,4,10
2,Hausa,Afro-Asiatic,haus1257,haus1257_REAL,N,0.58496,1.58496,2.25163,1.66667,5,5,12
3,Lebanese_Arabic,Afro-Asiatic,stan1323,stan1323_REAL,N,1.58219,2.57770,2.58219,1.00000,6,6,12
4,Old_Aramaic,Afro-Asiatic,olda1245,olda1245_REAL,N,1.00000,2.00000,2.00000,1.00000,5,4,10
...,...,...,...,...,...,...,...,...,...,...,...,...
1067,Tubar,Uto-Aztecan,tuba1279,tuba1279_REAL,N,0.00000,1.52193,-0.00000,-0.00000,3,1,3
1068,Tübatülabal,Uto-Aztecan,tuba1278,tuba1278_REAL,N,0.00000,0.91830,1.00000,1.00000,2,2,4
1069,Tümpisha_Shoshoni,Uto-Aztecan,pana1305,pana1305_REAL,N,0.88654,2.17574,2.58219,1.69565,6,6,16
1070,White_Mesa_Ute,Uto-Aztecan,utee1244b,utee1244b_REAL,N,0.00000,-0.00000,-0.00000,-0.00000,1,1,1


## Simulating kinship systems

To investigate whether kinship systems have higher mutual information than chance, we can build a random baseline for each language to serve as a point of comparison.

To do this, we will take each language in our dataset, and randomly scramble which terms go with which relatives (within generations). This will randomise the syncretisms within the paradigm, breaking any predictable structure built by the internal co-selection process, while maintaining the amount of variation across the system overall.
 
We take the following steps:

1. Extract the kinship system of a language from kinbank.
2. Filter the two generations we are interested in.
3. Randomly reassign the kinship terms to new types.
4. Repeat the process 1000 times for each language.

We already have the infrastructure for the first two! `get_kin_terms()`,  `get_pairs()` and `split_pairs()` will do this for us. So let's skip to 3, and write a function that randomises which terms form pairs, assuming that we have already extracted the kinship system and filtered the relevant pairs.

### 3. Randomly rearrange kin terms

`kintypes.py` also contains a list of which kin types are in which generation, which we can use to split a kinship system by generation.

In [17]:
def split_ks(ks):
    gn = {}
    gn1 = {}
    for entry in ks:
        if entry in kt.generation_n:
            gn[entry] = ks[entry]
        elif entry in kt.generation_n1:
            gn1[entry] = ks[entry]
        else:
            pass

    return gn,gn1        

Once the kinship system is split, we can then shuffle one of the generations and recombine the system.

In [18]:
def shuffle_ks(ks):
    gn_terms = []
    gn,gn1 = split_ks(ks)
    for term in gn:
        gn_terms.append(gn[term])

    random.shuffle(gn_terms)
    
    for i in range(len(gn)):
        # print(i)
        key = list(gn.keys())[i]
        ks[key] = gn_terms[i]

    return ks

If we split the kinship system, shuffle the G0 terms, and stick the system back together, then we can use `get_pairs()` just as we did for the real kinship systems to filter out the two generations we're interested in. From there, we can calculate MI for each simulation and save the data in a similar way as before.

In [19]:
def simulate_ks(ks: dict) -> list:
    simulation = shuffle_ks(ks)
    shuffled_pairs = get_pairs(simulation)
    return shuffled_pairs

In [20]:
def simulate_MI(families: list, filename: str, times: int):
    df = []
    codes = []
    filepath = '../../kinbank-family/'
    
    for family in tqdm(families):
        all_files = get_kb_files(filepath + family)
    
        for file in all_files:
            match = re.search('[a-z]{4}[0-9]{4}[a-z]?', file)
            code = match.group()
            language = file.split('_' + code)[0]

            if code not in codes:
                codes.append(code)

                ks = get_kin_terms(filepath + family + '/' + file)

                for i in range(times):
                    pairs = simulate_ks(ks)
                    if pairs:
                        results = {}
                        results['language'] = language
                        results['language_family'] = family
                        results['code'] = code
                        results['simulation_code'] = code + '_' + str(i)
                        results['simulation'] = 'Y'
                        write_data(pairs,results)

                        df.append(results)
    
    pd.DataFrame(df).to_csv('../data/raw/' + filename + '.csv',index=False)
    
    return pd.DataFrame(df)

In [21]:
# simulate_MI(['Indo-European'],'test',10)

In [21]:
# simulate_MI(families,'simulated_mi_UPDATED_NEW_KINTYPES',1000)

## Edit Distance

In this section we calculate the normalised Levenshtein edit distance between pairs of kin terms in each language and generate a monte carlo sample of edit distance for each language so that we can calculate the z score (as well as the correlation between edit distances).

To do this we need:
* A function to calculate the levenshtein distance between two forms.
* A function to calculate the distance between two meanings, which should have e.g. MB and MBD be closer than MZ and MBD.
* A function to calculate the correlation between a table of form distances and a table of meaning distances.
* A function to generate a monte carlo sample of correlations (ie the correlation of scrambled, simulated data).


UPDATE: we are not just comparing all labels for all relatives, but rather all labels for all categories! This prevents languages being penalised for being compositional but not having all unique labels across aunts and uncles.

So the first thing we need to do is work out which categories each language has, and who are the children of which category members.

In [27]:
test_lang = random.choice(all_kb_files)

test_ks = get_kin_terms('../../kinbank/' + test_lang)

g0,g1 = split_ks(test_ks)

In [48]:
def category_based_pairs(ks):
    g0,g1 = split_ks(ks)
    
    # work out which g1 relatives share a term
    
    relatives = list(g1.keys())
    terms = list(g1.values())

    categories = []

    for term in terms:
        cat = []
        indices = [index for index, element in enumerate(terms) if element == term]
        for index in indices:
            cat.append(relatives[index])
        categories.append(cat)

    categories = [list(i) for i in set(map(tuple, categories))]
    
    # work out which children should share a term on this basis
    
    child_cats = []

    for cat in categories:
        new_category = []
        for individual in cat:
            for pair in kt.ics_pairs:
                if pair[0] == individual:
                    new_category.append(pair[1])
        child_cats.append(new_category)
        
    # and make a new list of parent-child pairs that should have equal semantic distance
    
    new_pairs = []
    
    for i in range(len(categories)):
        new_pairs += list(itertools.product(categories[i], child_cats[i]))
        
    return new_pairs


In [49]:
from Levenshtein import distance as lvs_dist

In [50]:
def edit_distance(g0,g1):
    edit_dists = []

    for relative1 in g0:
        term1 = g0[relative1]
        for relative2 in g1:
            data = {}
            term2 = g1[relative2]
            
            if relative1 == relative2:
                pass
            
            else:
            
                if len(term1) > 0 and len(term2) > 0:
                    dist = lvs_dist(term1,term2)/len(max(term1,term2))
                    edit_dists.append(dist)
                else:
                    pass

    return edit_dists

To calculate semantic distance:

In [51]:
def semantic_distance(g0,g1): # (df,ks,pairs):
    sem_distances = []

    for relative1 in g0:
        for relative2 in g1:
            
            if relative1 == relative2:
                pass
            
            else:
            
                if len(g0[relative1]) > 0 and len(g1[relative2]) > 0:

                    if (relative1,relative2) in kt.ics_pairs or (relative2,relative1) in kt.ics_pairs:
                        distance = 1
                        sem_distances.append(distance)

                    else:
                        distance = 2
                        sem_distances.append(distance)

#     df['semantic_distance'] = sem_distances
#     
#     return df
    return sem_distances


The above will get us a meaning distance measure where if you are the child of a person, your meaning distance will be 1, and if you are not, your distance will be 2.

This captures the kind of compositionality in meaning we're interested in (e.g. that children of a person will have a similar form to that person) but not all kinds of kinship compositionality e.g. the Swedish situation.

Might have to do something more complex with features, e.g.
male vs female
mother side vs father side
young vs old
child-of vs not child-of

Anyway, for now we can calculate the correlation between our edit distance and our semantic distance with scipy.

But to get a z-score, we can just reuse our shuffling infrastructure from the original simulation (repeated here).

In [26]:
def shuffle_ks(ks):
    gn_terms = []
    gn,gn1 = split_ks(ks)
    for term in gn:
        gn_terms.append(gn[term])

    random.shuffle(gn_terms)
    
    for i in range(len(gn)):
        # print(i)
        key = list(gn.keys())[i]
        ks[key] = gn_terms[i]

    return ks

In [27]:
# def shuffle_distance(distances):
#     shuffled_edit_dists = distances.copy()
#     random.shuffle(shuffled_edit_dists)
#     return shuffled_edit_dists

And finally combine all of that into a single function that calculates the true correlation between edit distance and semantic distance for each language; simulates that language 1000 times and calculates the same correlation; and saves a z-score for each language.

In [28]:
def sample_distance(families,times,filename):
    output_df = []
    
    codes = []
    
    for family in families:
        files = get_kb_files('../../kinbank-family/' + family)
        for file in tqdm(files):
            
            match = re.search('[a-z]{4}[0-9]{4}[a-z]?', file)
            code = match.group()
            language = file.split('_' + code)[0]
            data = {}

            if code not in codes:
                codes.append(code)

                full_ks = get_kin_terms('../../kinbank/' + file)
                gn,gn1 = split_ks(full_ks)
#                 ks = {**gn,**gn1}
                

                edit = edit_distance(gn,gn1)
                sem = semantic_distance(gn,gn1)
        
                if len(edit) > 2:
                    true_corr = scipy.stats.pearsonr(np.array(edit),np.array(sem))[0]

                    sample_correlations = []
                    for i in range(times):
                        sample = shuffle_ks(full_ks)
                        sample_gn,sample_gn1 = split_ks(sample)
                        sample_edit = edit_distance(sample_gn,sample_gn1)
                        sample_sem = semantic_distance(sample_gn,sample_gn1)
                        sample_corr = scipy.stats.pearsonr(np.array(sample_edit),np.array(sample_sem))[0]
                        sample_correlations.append(sample_corr)

#                     print(sample_correlations)
                    mean = np.mean(sample_correlations)
                    print(mean)
                    sd = np.std(sample_correlations)
                    print(sd)
                    z = (true_corr - mean) / sd
                    print(z)

                    data['language'] = language
                    data['family'] = family
                    data['code'] = code
                    data['correlation'] = true_corr
                    data['mean'] = mean
                    data['sd'] = sd
                    data['z'] = z

                    output_df.append(data)
                
    pd.DataFrame(output_df).to_csv('../data/raw/' + filename + '.csv',index=False)

    return pd.DataFrame(output_df)
                    

In [32]:
# sample_distance(['Indo-European'],10,'TEST')

In [34]:
# sample_distance(families, 1000, 'edit_distance_UPDATED_SEMD')

 17%|██████████████                                                                      | 1/6 [00:00<00:02,  1.92it/s]

-0.28630209319690514
5.903526765724995e-17
0.9403048962790747


 33%|████████████████████████████                                                        | 2/6 [00:02<00:06,  1.53s/it]

-0.02157702239239466
0.027620452963079734
7.1179382270513685


 50%|██████████████████████████████████████████                                          | 3/6 [00:05<00:05,  1.87s/it]

...

 98%|████████████████████████████████████████████████████████████████████████████████▏ | 45/46 [00:28<00:00,  1.67it/s]

0.0013978144331980933
0.03112702990591622
-3.350962503655467
0.6324555320336758
1.1102230246251565e-16
1.0


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:29<00:00,  1.57it/s]

0.14485667881883144
0.013412405894290628
1.4247236653831115


,language,family,code,correlation,mean,sd,z
0,Akkadian,Afro-Asiatic,akka1240,-0.286302,-0.286302,5.903527e-17,0.940305
1,Amharic,Afro-Asiatic,amha1245,0.175024,-0.021577,2.762045e-02,7.117938
2,Hausa,Afro-Asiatic,haus1257,-0.023459,-0.013742,1.202095e-02,-0.808326
3,Lebanese_Arabic,Afro-Asiatic,stan1323,0.061818,-0.001733,3.403751e-02,1.867088
4,Old_Aramaic,Afro-Asiatic,olda1245,0.118169,-0.052229,3.368251e-02,5.058931
...,...,...,...,...,...,...,...
1065,Tubar,Uto-Aztecan,tuba1279,-0.359856,-0.359856,1.110223e-16,1.000000
1066,Tübatülabal,Uto-Aztecan,tuba1278,0.098295,0.098295,1.387779e-17,-1.000000
1067,Tümpisha_Shoshoni,Uto-Aztecan,pana1305,-0.102908,0.001398,3.112703e-02,-3.350963
1068,White_Mesa_Ute,Uto-Aztecan,utee1244b,0.632456,0.632456,1.110223e-16,1.000000
